In [1]:
import torch
from datasets import load_dataset
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import f1_score

MODEL_NAME = "xlm-roberta-base"
LABEL_NAMES =  ["gender", "regional", "caste", "religion", "appearance", "socialstatus", "ambiguity", "political", "age", "disability", "citizenship", "authority"]
NUM_LABELS = len(LABEL_NAMES)


dataset = load_dataset("json", data_files={"train": "nepali_social_bias_10000_varied_tone.jsonl"})

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)

def preprocess(example):
    encoding = tokenizer(
        example["text"],
        truncation=True,
        max_length=256
    )
    encoding["labels"] = [float(example[label]) for label in LABEL_NAMES]
    return encoding

dataset = dataset.map(
    preprocess,
    remove_columns=dataset["train"].column_names
)

dataset.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# Freeze encoder (low compute)
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    labels = labels.astype(int)

    return {
        "f1_micro": f1_score(labels, preds, average="micro"),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

training_args = TrainingArguments(
    output_dir="./bias_model",
    per_device_train_batch_size=8,
    num_train_epochs=30,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("./bias_model")
tokenizer.save_pretrained("./bias_model")

print("Fine-tuning completed and model saved.")


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3829722007.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.566800
20,0.373900
30,0.318000
40,0.311900
50,0.306000
60,0.302200
70,0.302700
80,0.301800
90,0.311700
100,0.305500


Fine-tuning completed and model saved.


In [6]:
!zip -r checkpoint_full.zip /content/bias_model/checkpoint-37500

  adding: content/bias_model/checkpoint-37500/ (stored 0%)
  adding: content/bias_model/checkpoint-37500/special_tokens_map.json (deflated 52%)
  adding: content/bias_model/checkpoint-37500/tokenizer_config.json (deflated 76%)
  adding: content/bias_model/checkpoint-37500/training_args.bin (deflated 53%)
  adding: content/bias_model/checkpoint-37500/config.json (deflated 59%)
  adding: content/bias_model/checkpoint-37500/scheduler.pt (deflated 61%)
  adding: content/bias_model/checkpoint-37500/optimizer.pt (deflated 7%)
  adding: content/bias_model/checkpoint-37500/trainer_state.json (deflated 85%)
  adding: content/bias_model/checkpoint-37500/model.safetensors (deflated 42%)
  adding: content/bias_model/checkpoint-37500/rng_state.pth (deflated 26%)
  adding: content/bias_model/checkpoint-37500/sentencepiece.bpe.model (deflated 49%)


In [7]:
from google.colab import files
files.download('checkpoint_full.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>